In [1]:
#load "Paket.fsx"
Paket.Package
  [ "QBitNinja.Client -Version 1.0.3.21"
    "NBitcoin -Version 3.0.0.33"
    "FSharp.Data -Version 2.4.6"
    "Newtonsoft.Json -Version 10.0.3"    
  ]    
#load "Paket.Generated.Refs.fsx"

open System
open NBitcoin
open NBitcoin.Crypto   
open NBitcoin.OpenAsset 
open NBitcoin.Stealth 
open QBitNinja.Client 
open QBitNinja.Client.Models 
open System.Collections.Generic 
open System.Linq
open System.Collections


type Wallet =
    static member PrivateKey: Key = 
        (new Key())
    static member PublicKey (k:Key) = 
        if (k=null) then
          (new Key()).PubKey
        else
          k.PubKey 
          
type Wallet with
     static member BitcoinSecretMain (k:Key) = 
            if (k=null) then
                Wallet.PrivateKey.GetBitcoinSecret(Network.Main)
            else
                k.GetBitcoinSecret(Network.Main)
     static member BitcoinSecretTest (k:Key) = 
            if (k=null) then
                Wallet.PrivateKey.GetBitcoinSecret(Network.TestNet)
            else
                k.GetBitcoinSecret(Network.TestNet)    
     static member WifMain (k:Key) = 
            if (k=null) then
                Wallet.PrivateKey.GetWif(Network.Main)
            else
                k.GetWif(Network.Main)
     static member WifTest (k:Key) = 
            if (k=null) then
                Wallet.PrivateKey.GetWif(Network.TestNet)
            else
                k.GetWif(Network.TestNet)
type Wallet with
     static member BitCoinAddressMain (k:Key) = 
        Wallet.PublicKey(k).GetAddress(Network.Main) 
     static member BitCoinAddressTest  (k:Key) = 
        Wallet.PublicKey(k).GetAddress(Network.TestNet) 
     static member PublicKeyHash (k:Key) = 
        Wallet.PublicKey(k).Hash 
     static member BitCoinAddressMain2 (k:Key) = 
        Wallet.PublicKey(k).Hash.GetAddress(Network.Main) 
     static member BitCoinAddressTest2  (k:Key) = 
        Wallet.PublicKey(k).Hash.GetAddress(Network.TestNet) 
        
type Script = 
   static member PublicKeyHash (Pk:string) = 
       new KeyId(Pk) 
   static member Main  (Pk:string) = 
       Script.PublicKeyHash(Pk).GetAddress(Network.Main).ScriptPubKey
   static member Test  (Pk:string) = 
       Script.PublicKeyHash(Pk).GetAddress(Network.TestNet).ScriptPubKey 
   static member Main  (k:Key)  = 
       Wallet.PublicKeyHash(k).GetAddress(Network.Main).ScriptPubKey 
   static member Test  (k:Key)  = 
       Wallet.PublicKeyHash(k).GetAddress(Network.TestNet).ScriptPubKey 
   static member BitCoinAddressMain (k:Key) = 
      Script.Main(k).GetDestinationAddress(Network.Main)
   static member BitCoinAddressTest  (k:Key) = 
      Script.Main(k).GetDestinationAddress(Network.TestNet)

In [2]:
open FSharp.Data
open FSharp.Data.JsonExtensions

type secretJson = JsonProvider<"""{
  "secret": ""
}""">
      
let sj = secretJson.GetSample()
//printfn "%A" sj 
  
let json_uri = @"/home/nbuser/library/secret.json"
let wwif = secretJson.Load(json_uri).JsonValue?secret.AsString()  

let network = Network.TestNet

let sender_bitcoinPrivateKey:Key  = Key.Parse(wwif, Network.TestNet) 
let sender_address = Script.BitCoinAddressTest sender_bitcoinPrivateKey //Wallet.PublicKeyHash bitcoinPrivateKey

printfn "%A" (network)
printfn "sender %A" (sender_address)
printfn "sender secret %A" (Wallet.BitcoinSecretTest sender_bitcoinPrivateKey)
printfn "sender script %A" (Script.Test sender_bitcoinPrivateKey)

let json_uri_dest = @"/home/nbuser/library/secret_dest.json"
let wwif_dest = secretJson.Load(json_uri_dest).JsonValue?secret.AsString()  


let dest_bitcoinPrivateKey:Key   = Key.Parse(wwif_dest, Network.TestNet) 
let dest_address = Script.BitCoinAddressTest dest_bitcoinPrivateKey  
printfn "destination %A" (dest_address)
printfn "dest secret %A" (Wallet.BitcoinSecretTest dest_bitcoinPrivateKey)
printfn "dest script %A" (Script.Test dest_bitcoinPrivateKey)


TestNet
sender mnnaUEKBKJVFjCqVnVPmwefk6CwuVsyRRj
sender secret cRMR4cGGGCh67WmHnZk7Myy9P4nrEuedRovrD3jSqHg7K3HY5wmv
sender script OP_DUP OP_HASH160 4fbd5152456bee3dc0ffe25118a6de17126e5c19 OP_EQUALVERIFY OP_CHECKSIG
destination mtYy84R9Ezc6UxWGfHgMj4TmxXFqSy7SPf
dest secret cVdNBTECyg4Af4reyg4qykBMTq8SkqNboJRKotAvEMdVKnhr3CaK
dest script OP_DUP OP_HASH160 8efb2874941ae0a85e2c94ee6b267e8ad70fb5cf OP_EQUALVERIFY OP_CHECKSIG


In [3]:
let client:QBitNinjaClient  = new QBitNinjaClient(Network.TestNet)

let transactionId = uint256.Parse("06927bb3aaa2d1800548105d97a4f40ff61a68821f249a90158ddb76b082a7e8")

// Query the transaction
let transactionResponse:GetTransactionResponse  = client.GetTransaction(transactionId).Result

let transaction:NBitcoin.Transaction  = transactionResponse.Transaction

let receivedCoins = transactionResponse.ReceivedCoins;
let mutable myCoins:Coin = null 
for coin in receivedCoins do
    if (coin.TxOut.ScriptPubKey =  sender_bitcoinPrivateKey.ScriptPubKey) then
        myCoins<- coin:?>Coin

printfn "%A" myCoins.Amount

 

0.52980000


In [4]:
open System
open System.IO
open System.Security.Cryptography

let  GetStream (s:string) =
                let stream = new MemoryStream()
                let writer = new StreamWriter(stream)
                writer.Write(s)
                writer.Flush()
                stream.Position <- 0L
                stream
                
//Print the byte array in a readable format.
let ByteString (b:byte[]) = 
        let mutable i = 0
        let mutable s:string = ""
        for i = 0  to b.Length - 1 do
            s <- s + String.Format("{0:X2}", b.[i])
            if (i % 4 = 3) then
                s <- s+ " "
        s    
        
let ByteToHex bytes = 
    bytes 
    |> Array.map (fun (x : byte) -> System.String.Format("{0:X2}", x))
    |> String.concat System.String.Empty

let message = """{
  "formazione": {
    "id_competizione": 1,
    "id_lega": 1,
    "id_squadra": 1,
    "id_calendario": 30,
    "titolari": [
      ALISON,
      CACCIATORE,
      DE SILVESTRI,
      SILVESTRE,
      VICARI,
      BENASSI,
      CRISTANTE,
      HETEMAJ,
      LINETTY,
      QUAGLIARELLA,
      INGLESE
    ],
    "panchina": [
      SKORUPSKI,
      GALABINOV,
      MATOS,
      THEREAU,
      SIMY,
      LAZZARIM.,
      STROOTMAN,
      GILDIAS,
      MIRANDA,
      NUYTINCK,
      TOMOVIC,
      BASTOS
    ],
    "giornata_lega": 30,
    "giornata_a": 32,
    "modulo": "4-4-2",
    "data": 1523911213
  }
}"""

let Sha256 (doc:string) = 
    let sdoc = GetStream doc
    let sh = new SHA256Managed()
    sh.ComputeHash(sdoc)

let hashValue = Sha256 message    

printfn "%A" hashValue

[|206uy; 242uy; 188uy; 42uy; 64uy; 151uy; 118uy; 163uy; 148uy; 19uy; 30uy; 114uy;
  185uy; 230uy; 135uy; 20uy; 100uy; 167uy; 209uy; 76uy; 166uy; 234uy; 172uy;
  240uy; 212uy; 254uy; 133uy; 55uy; 1uy; 182uy; 213uy; 55uy|]


In [5]:
 
let mutable txBuilder = new TransactionBuilder() 
let mutable tx:Transaction = txBuilder
                                .AddCoins(myCoins) 
                                .AddKeys(sender_bitcoinPrivateKey)
                                .Send(dest_address, new Money(1m, MoneyUnit.Satoshi))
                                .SetChange(sender_address)
                                .SendFees(new Money(0.001m, MoneyUnit.BTC))
                                .Send(TxNullDataTemplate.Instance.GenerateScriptPubKey(hashValue), Money.Zero)
                                .BuildTransaction(true)

printfn "%A" tx

{
  "hash": "db3fd1a93c03ea4763a39c1453366ffcb4acb6258ea58a04a4aa27f0501d7581",
  "ver": 1,
  "vin_sz": 1,
  "vout_sz": 2,
  "lock_time": 0,
  "size": 235,
  "in": [
    {
      "prev_out": {
        "hash": "06927bb3aaa2d1800548105d97a4f40ff61a68821f249a90158ddb76b082a7e8",
        "n": 0
      },
      "scriptSig": "3045022100d6376bdf42eb21d8c716c2c59533caacc910734072aa5e20e1f75f1d49c17dde02204a5f9f8e5652aa9487ca66f48399ae241cbc5b16b7f6a618a6f9a5119a52c33d01 0364dfe2fa8c584d9af693fca129f9c031e4108be2b6d31ac3f6846ef1ba7ecdc1"
    }
  ],
  "out": [
    {
      "value": "0.52879999",
      "scriptPubKey": "OP_DUP OP_HASH160 4fbd5152456bee3dc0ffe25118a6de17126e5c19 OP_EQUALVERIFY OP_CHECKSIG"
    },
    {
      "value": "0.00000000",
      "scriptPubKey": "OP_RETURN cef2bc2a409776a394131e72b9e6871464a7d14ca6eaacf0d4fe853701b6d537"
    }
  ]
}


In [6]:
let verify = txBuilder.Verify(tx)  //check fully signed
printfn "%A" (verify)

true
